In [1]:
!pip install --upgrade pip
!pip install --upgrade selenium
#!conda install -c conda-forge webdriver-manager
!pip install osascript

  Using cached pip-23.3.1-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 21.2.3
    Uninstalling pip-21.2.3:
      Successfully uninstalled pip-21.2.3
  Using cached urllib3-2.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached attrs-23.1.0-py3-none-any.whl (61 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 500.5 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 1.3 MB/s eta 0:00:0000:010:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.5/162.5 kB 864.1 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.3/448.3 kB 3.8 MB/s eta 0:00:0000:0100:01
Using cached urllib3-2.1.0-py3-none-any.whl (104 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 777.2 kB/s eta 0:00:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting require

In [5]:
%%capture
!pip install requests
!pip install bs4

In [22]:
!pip install urllib3

In [7]:
import re
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
#from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import json

import os
import time
import osascript

from urllib.parse import unquote



In [52]:
#url='https://www.bestlawyers.com/search/best-lawyers/us'
url = 'https://www.bestlawyers.com/search?pub-list-id=1&cc=US&page=1&sort=relevancy&lawyers=1'


In [53]:
service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)
driver.get(url)
driver.maximize_window()

In [55]:
def extract_location_from_google_maps_link(google_maps_link):

    decoded_link = unquote(google_maps_link)

    start_index = decoded_link.find("/search/") + len("/search/")
    location_text = decoded_link[start_index:]
    
    location_text = location_text.replace("+", " ").replace("%2c", ",")

    return location_text

In [76]:
lawyers_df = pd.DataFrame(columns=['Name', 'Website', 'Address', 'Phone', 'Firm'])
data = {
    'names' : [],
    'websites' : [],
    'addresses' : [],
    'phones' : [],
    'firms' : []
}


for i in range(1,21):
    next_url = f'https://www.bestlawyers.com/search?pub-list-id=1&cc=US&page={i}&sort=relevancy&lawyers=1'
    driver.get(next_url)

    #WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="grid_ce05a831aaef47c29a0f6563b79c94f0"]')))
    time.sleep(2)
    
    html  = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    cards = soup.find_all('div', {'class':['card-scaler']})

    for k, card in enumerate(cards):

        
        lawyer_url = 'https://www.bestlawyers.com/'+card.find('a')['href']
        
        #### LAWYER URL ####
        driver.get(lawyer_url)
        time.sleep(1)
        
        html_lawyer = driver.page_source
        soup_lawyer = BeautifulSoup(html_lawyer, 'html.parser')
        #WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/main/div[1]/div[1]/div[1]/div[1]/h1/a")))
        
        
        #### NAME ####
        try:
            name = soup_lawyer.find('h1', class_='lawyer-name').text.strip()
            #name = driver.find_element(By.XPATH, '/html/body/main/div[1]/div[1]/div[1]/div[1]/h1/a').text
            data['names'].append(name)
        except:
            print(f'Error al buscar name para {name}. Index {k}')
            data['name'].append('')


        #### WEBSITE ####
        try:
            website = soup_lawyer.find('a', class_ = 'ga-lawyer-link')['href']
            data['websites'].append(website)
        except:
            print(f'Error al buscar website para {name}. Index {k}')
            data['websites'].append('')

        
        #### ADDRESS ####
        try:
            gmap_link = soup_lawyer.find('a', {'title': 'See map location'})['href']
            address = extract_location_from_google_maps_link(gmap_link)
            data['addresses'].append(address)
        except:
            print(f'Error al buscar address para {name}. Index {k}')
            data['addresses'].append('')

        
        #### PHONE ####
        try:
            phone = soup_lawyer.find('a', class_ = 'ga-lawyer-phone')['href'][4:]
            data['phones'].append(phone)
        except:
            print(f'Error al buscar phone para {name}. Index {k}')
            data['phones'].append('')


        #### FIRM ####
        try:
            firm = driver.find_element(By.XPATH, '/html/body/main/div[1]/div[1]/div[1]/div[1]/p[1]/a').text
            data['firms'].append(firm)
        except:
            print(f'Error al buscar firm para {name}. Index {k}')
            data['firms'].append('')
        
        
        
        print(f'Done with {name}!')
        driver.back()
        #print(data)
        time.sleep(1)

    print('\n=====Sleeping before next lawyers...=====\n')    
    time.sleep(2)



actions = ActionChains(driver)
actions.move_to_element(driver.find_element(By.XPATH, '//*[@id="grid_6b709172227b4fd0ae04aee78cb86ff9"]/div[4]/div/button[1]')).perform()
        

Error al buscar phone para Tara Aaron-Stelluto. Index 0
Done with Tara Aaron-Stelluto!
Done with William D. "Bill" Aaron, Jr.!
Error al buscar phone para Mark J. Abate. Index 2
Done with Mark J. Abate!
Done with Donald R. "Don" Abaunza!
Error al buscar phone para Carolyn Abbate. Index 4
Done with Carolyn Abbate!
Done with Allison Abbott!
Error al buscar phone para Charles H. Abbott. Index 6
Done with Charles H. Abbott!
Error al buscar phone para Hirschel T. Abbott, Jr.. Index 7
Done with Hirschel T. Abbott, Jr.!
Done with Scott M. Abbott!
Error al buscar phone para Susan L. "Sue" Abbott. Index 9
Done with Susan L. "Sue" Abbott!
Error al buscar phone para Michael V. "Mike" Abcarian. Index 10
Done with Michael V. "Mike" Abcarian!
Error al buscar phone para Mary Clift Abdalla. Index 11
Done with Mary Clift Abdalla!
Error al buscar phone para Amy Abdo. Index 12
Done with Amy Abdo!
Error al buscar phone para George E. Abdo III. Index 13
Done with George E. Abdo III!
Error al buscar phone pa

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="grid_6b709172227b4fd0ae04aee78cb86ff9"]/div[4]/div/button[1]"}
  (Session info: chrome=119.0.6045.199); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000102dcdd28 chromedriver + 4795688
1   chromedriver                        0x0000000102dc52b3 chromedriver + 4760243
2   chromedriver                        0x000000010299e88d chromedriver + 407693
3   chromedriver                        0x00000001029ed3d9 chromedriver + 730073
4   chromedriver                        0x00000001029ed591 chromedriver + 730513
5   chromedriver                        0x0000000102a337e4 chromedriver + 1017828
6   chromedriver                        0x0000000102a14ccd chromedriver + 892109
7   chromedriver                        0x0000000102a3

In [79]:
lawyers_df['Name'] = data['names']
lawyers_df['Website'] = data['websites']
lawyers_df['Address'] = data['addresses']
lawyers_df['Phone'] = data['phones']
lawyers_df['Firm'] = data['firms']

In [84]:
lawyers_df[lawyers_df['Phone']=='']

,Name,Website,Address,Phone,Firm
0,Tara Aaron-Stelluto,https://www.bartonesq.com/attorney/tara-aaron-...,"611 Commerce Street, Suite 2603 Nashville TN US",,Barton LLP
2,Mark J. Abate,https://www.goodwinlaw.com/en/people/a/abate-mark,The New York Times Building 620 Eighth Avenue ...,,Goodwin
4,Carolyn Abbate,https://vafamilylaw.com/carolyn-m-abbate/,649 South Washington Street Alexandria VA US,,"Grenadier, Duffett, Levi, Winkler & Rubin, P.C."
6,Charles H. Abbott,https://www.formanwatkins.com/people/charles-h...,"201 St. Charles Avenue, Suite 2100 New Orleans...",,Forman Watkins & Krutz LLP
7,"Hirschel T. Abbott, Jr.",https://www.stonepigman.com/attorneys-Hirschel...,"909 Poydras Street, Suite 3150 New Orleans LA US",,Stone Pigman Walther Wittmann L.L.C.
...,...,...,...,...,...
315,Craig Y. Allison,https://www.dickinson-wright.com/our-people/cr...,"424 Church Street, Suite 800 Nashville TN US",,Dickinson Wright PLLC
316,Evan Allison,https://millermartin.com/people/Attorney/Evan-...,"Volunteer Building, Suite 1200 \r\n832 Georgia...",,Miller & Martin PLLC
317,Grant F. Allison,https://legalhawaii.com/attorneys/grant-f-alli...,"Topa Financial Center, Suite 900 700 Bishop St...",,Lung Rose Voss & Wagnild
318,John D. Allison,https://www.klgates.com/John-D-Allison,"300 South Tryon Street, Suite 1000 Charlotte N...",,K&L Gates LLP


In [89]:
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 1.9 MB/s eta 0:00:0000:0100:01


In [91]:
lawyers_df.to_excel('best_lawyers.xlsx', index=False)

In [12]:
lawyers_df = pd.DataFrame(columns=['Name', 'Website', 'Address', 'Phone', 'Firm'])
data = {
    'names' : [],
    'webistes' : [],
    'addresses' : [],
    'phones' : [],
    'firms' : []
}

k = 1
for i in range(0,20):
   
    lawyer_card = f'//*[@id="carousel{i}"]/div[2]/div[1]/div'
    print(lawyer_card)
    
    #actions = ActionChains(driver)
    #actions.move_to_element(driver.find_element(By.XPATH, lawyer_card)).perform()
    #element_to_move = driver.find_element(By.XPATH, lawyer_card)
    #actions = ActionChains(driver)
    #actions.move_to_element(element_to_move)
    #time.sleep(2)
    #actions.move_by_offset(25, 0)
    #actions.perform()


    
    time.sleep(2)
   # WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, lawyer_card)))
    #print(soup.find('div', class_='card-lawyer').find('div', class_='position-relative').find('a', class_='position-absolute')['href'])
    url_lawyer = 'https://www.bestlawyers.com/'+soup.find('div', class_='card-lawyer').find('div', class_='position-relative').find('a', class_='position-absolute')['href']
    driver.get(url_lawyer)
    time.sleep(1)
    #break
   # driver.find_element(By.XPATH, lawyer_card).click()

    #### NAME ####
    html_lawyer = driver.page_source
    soup_lawyer = BeautifulSoup(html_lawyer, 'html.parser')
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/main/div[1]/div[1]/div[1]/div[1]/h1/a")))
    name = driver.find_element(By.XPATH, '/html/body/main/div[1]/div[1]/div[1]/div[1]/h1/a').text
    data['names'].append(name)

    #### WEBSITE ####
    try:
       website = soup_lawyer.find('a', class_ = 'ga-lawyer-link')['href']
       data['webistes'].append(website)
    except:
        print(f'Error al buscar website para {name}. Index {k}')

    
    #### ADDRESS ####
    try:
        gmap_link = soup_lawyer.find('a', {'title': 'See map location'})['href']
        address = extract_location_from_google_maps_link(gmap_link)
        data['addresses'].append(address)
    except:
        print(f'Error al buscar address para {name}. Index {k}')

    
    #### PHONE ####
    try:
       phone = soup_lawyer.find('a', class_ = 'ga-lawyer-phone')['href'][4:]
       data['phones'].append(phone)
    except:
        print(f'Error al buscar phone para {name}. Index {k}')


    #### FIRM ####
    firm = driver.find_element(By.XPATH, '/html/body/main/div[1]/div[1]/div[1]/div[1]/p[1]/a').text
    data['firms'].append(firm)
    
    print(f'Done with {name}!')
    driver.back()
    #print(data)
    time.sleep(1)

//*[@id="carousel0"]/div[2]/div[1]/div
Error al buscar phone para Tara Aaron-Stelluto. Index 0
Done with Tara Aaron-Stelluto!
//*[@id="carousel1"]/div[2]/div[1]/div
Error al buscar phone para Tara Aaron-Stelluto. Index 1
Done with Tara Aaron-Stelluto!


KeyboardInterrupt: 

In [37]:
data

{'names': ['Tara Aaron-Stelluto',
  'William D. "Bill" Aaron, Jr.',
  'Mark J. Abate',
  'Donald R. "Don" Abaunza'],
 'webistes': ['https://www.bartonesq.com/attorney/tara-aaron-stelluto/',
  'https://www.aaronfirm.com/aaron.html',
  'https://www.goodwinlaw.com/en/people/a/abate-mark',
  'https://www.liskow.com/team/donald-r-abaunza/'],
 'addresses': ['611 Commerce Street, Suite 2603 Nashville TN US',
  '201 Saint Charles Avenue, Suite 3800 New Orleans LA US',
  'The New York Times Building 620 Eighth Avenue New York NY US',
  'Hancock Whitney Center, Suite 5000 701 Poydras Street New Orleans LA US'],
 'phones': ['5045691800', '5045817979'],
 'firms': ['Barton LLP', 'Aaron & Gianna, PLC', 'Goodwin', 'Liskow & Lewis']}

In [5]:
driver.find_element(By.XPATH, '//*[@id="__next"]/div[3]/main/div/div[2]/div/div[1]/div/div[1]/div[1]/div/a').click()

In [6]:
html_page = driver.page_source
soup_page = BeautifulSoup(html_page, 'html.parser')

In [7]:
element = driver.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/div/div/div[2]/div/div[1]/div[2]/div[2]/button[1]/div')

In [8]:
element.click()

In [9]:
driver.switch_to.window(driver.window_handles[1])

In [10]:
driver.current_url

'https://pitch.com/?utm_medium=referral&utm_source=producthunt&utm_campaign=png&ref=producthunt'

In [77]:
html_new = driver.page_source

In [78]:
soup_new = BeautifulSoup(html_new, 'html.parser')

In [35]:
full_json = json.loads(soup_page.find_all('script', id='__NEXT_DATA__')[0].text)

In [46]:
'websiteUrl' in soup_page.find_all('script', id='__NEXT_DATA__')[0].text

False

In [38]:
data_json = full_json['props']['apolloState']

In [41]:
data_json#['Product111551']

{'Topic44': {'__typename': 'Topic', 'id': '44', 'slug': 'design-tools'},
 'Topic46': {'__typename': 'Topic', 'id': '46', 'slug': 'productivity'},
 'Topic352': {'__typename': 'Topic', 'id': '352', 'slug': 'tech'},
 'FundingSurvey42248': {'__typename': 'FundingSurvey',
  'id': '42248',
  'bootstrapped': False},
 'Product111551': {'__typename': 'Product',
  'id': '111551',
  'slug': 'pitch',
  'name': 'Pitch',
  'postsCount': 9,
  'isSubscribed': False},
 'Post420125': {'__typename': 'Post',
  'id': '420125',
  'hideVotesCount': False,
  'commentsCount': 40,
  'name': 'Pitch 2.0',
  'shortenedUrl': '/r/p/420125',
  'slug': 'pitch-2-0',
  'tagline': 'Create, share, and analyze presentations faster than ever',
  'updatedAt': '2023-11-14T05:17:54-08:00',
  'topics({"first":4})': {'__typename': 'TopicConnection',
   'edges': [{'__typename': 'TopicEdge', 'node': {'__ref': 'Topic44'}},
    {'__typename': 'TopicEdge', 'node': {'__ref': 'Topic46'}},
    {'__typename': 'TopicEdge', 'node': {'__ref

In [5]:
data = pd.DataFrame(columns=['Business', 'UBI', 'Address', 'City', 'ZIP', 'Eff_Date', 'Exp_Date'])

In [47]:
import requests

cookies = {
    'first_visit': '1699967938',
    'first_referer': '',
    '_delighted_web': '{%2271AaKmxD4TpPsjYW%22:{%22_delighted_fst%22:{%22t%22:%221699967939047%22}}}',
    '_ga': 'GA1.1.1451722093.1699967939',
    'ajs_anonymous_id': 'be82d70a-8b59-48d7-bec6-8db5bd17f147',
    'visitor_id': '3b2d7527-6312-46c2-8dd5-b647421b4311',
    'track_code': '837c10afc1',
    '__hstc': '43932346.ae3b96c9b993df067f87fe90e82eb53a.1699967939606.1699967939606.1699967939606.1',
    'hubspotutk': 'ae3b96c9b993df067f87fe90e82eb53a',
    '__hssrc': '1',
    '__hssc': '43932346.1.1699967939606',
    'intercom-id-fe4ce68d4a8352909f553b276994db414d33a55c': '09750395-26ce-4b02-80b9-a402db0a6844',
    'intercom-session-fe4ce68d4a8352909f553b276994db414d33a55c': '',
    'intercom-device-id-fe4ce68d4a8352909f553b276994db414d33a55c': 'af535af4-3e04-4750-826c-96ac2f174a32',
    '_ga_WZ46833KH9': 'GS1.1.1699967939.1.1.1699968282.59.0.0',
    'csrf_token': '5BKlvhO3DqHuD5Y3PutaO0TUVfOufS8NGYPloYbo-XfAQoP5uEFEicboWxIaW7LzKDszBNofixs40yCQvc4LLQ',
    '_producthunt_session_production': 'RzdK5QwJAfo%2B%2FoL2ekhwZyusVvMhkE6If0H6eB%2FRdAI5V%2BqfJ%2B%2BhoDn2k%2FfVlAqWw89%2FEeJhCxmICH%2Frk%2B9qcHHeA3AAqNGiM48%2B0D6QSDS2NSHCEMsrTyiT%2Bu9sjD2MoQmvfZbYEQD%2B5XyJUFHU%2BN6VlrXrZucqqZ%2F3FAohe3Au9054Lrt71P3fONejw3IHKbGv2n03n5fXxLcTaH2lnYiOk5rAltnBLnQNp9vV6RqFqD5fL9CDdk1Ox7fSdiIB2na0ztKMRIvaWqRp4UGcMGhGTMWMrNyylG99sBOIgBT06lkQVZqsqlusInmV0G3LvVmybrKIuLCBbdxzwA%3D%3D--hqII%2FnkfWPWp6bRa--IMUYGDRYAoQl08Zf3qExWg%3D%3D',
}

headers = {
    'authority': 'www.producthunt.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'es-419,es;q=0.9',
    'cache-control': 'max-age=0',
    # 'cookie': 'first_visit=1699967938; first_referer=; _delighted_web={%2271AaKmxD4TpPsjYW%22:{%22_delighted_fst%22:{%22t%22:%221699967939047%22}}}; _ga=GA1.1.1451722093.1699967939; ajs_anonymous_id=be82d70a-8b59-48d7-bec6-8db5bd17f147; visitor_id=3b2d7527-6312-46c2-8dd5-b647421b4311; track_code=837c10afc1; __hstc=43932346.ae3b96c9b993df067f87fe90e82eb53a.1699967939606.1699967939606.1699967939606.1; hubspotutk=ae3b96c9b993df067f87fe90e82eb53a; __hssrc=1; __hssc=43932346.1.1699967939606; intercom-id-fe4ce68d4a8352909f553b276994db414d33a55c=09750395-26ce-4b02-80b9-a402db0a6844; intercom-session-fe4ce68d4a8352909f553b276994db414d33a55c=; intercom-device-id-fe4ce68d4a8352909f553b276994db414d33a55c=af535af4-3e04-4750-826c-96ac2f174a32; _ga_WZ46833KH9=GS1.1.1699967939.1.1.1699968282.59.0.0; csrf_token=5BKlvhO3DqHuD5Y3PutaO0TUVfOufS8NGYPloYbo-XfAQoP5uEFEicboWxIaW7LzKDszBNofixs40yCQvc4LLQ; _producthunt_session_production=RzdK5QwJAfo%2B%2FoL2ekhwZyusVvMhkE6If0H6eB%2FRdAI5V%2BqfJ%2B%2BhoDn2k%2FfVlAqWw89%2FEeJhCxmICH%2Frk%2B9qcHHeA3AAqNGiM48%2B0D6QSDS2NSHCEMsrTyiT%2Bu9sjD2MoQmvfZbYEQD%2B5XyJUFHU%2BN6VlrXrZucqqZ%2F3FAohe3Au9054Lrt71P3fONejw3IHKbGv2n03n5fXxLcTaH2lnYiOk5rAltnBLnQNp9vV6RqFqD5fL9CDdk1Ox7fSdiIB2na0ztKMRIvaWqRp4UGcMGhGTMWMrNyylG99sBOIgBT06lkQVZqsqlusInmV0G3LvVmybrKIuLCBbdxzwA%3D%3D--hqII%2FnkfWPWp6bRa--IMUYGDRYAoQl08Zf3qExWg%3D%3D',
    'sec-ch-ua': '"Google Chrome";v="119", "Chromium";v="119", "Not?A_Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
}

params = {
    'r': 'h',
}

response = requests.get('https://www.producthunt.com/posts/pitch-2-0', params=params, cookies=cookies, headers=headers)

'<script id="__NEXT_DATA__" type="application/json">{"props":{"pageProps":{"type":"loaded","routingProps":{"params":{},"location":{"pathname":"/","query":{},"search":"","hash":""}},"pageQueryVariables":{"cursor":null,"kind":"FEATURED","filters":{}},"_sentryTraceData":"3a9cd6be3f0249eb9ed0995330e88316-9f8643be149a33d2-0","_sentryBaggage":"sentry-environment=production,sentry-release=K77Jpoq1o8Vil1RvwKrJp,sentry-public_key=58db488a6c2941a28833ea5ca522361e,sentry-trace_id=3a9cd6be3f0249eb9ed0995330e88316,sentry-sample_rate=0,sentry-transaction=%2F,sentry-sampled=false"},"serverSideRendered":true,"apolloState":{"Topic44":{"__typename":"Topic","id":"44","slug":"design-tools"},"Topic46":{"__typename":"Topic","id":"46","slug":"productivity"},"Topic352":{"__typename":"Topic","id":"352","slug":"tech"},"FundingSurvey42248":{"__typename":"FundingSurvey","id":"42248","bootstrapped":false},"Product111551":{"__typename":"Product","id":"111551","slug":"pitch","name":"Pitch","postsCount":9,"isSubscrib

In [58]:
script_html = str(soup_page.find_all('script', id='__NEXT_DATA__')[0])
# clean up the HTML
script_html = script_html.replace('\n', ' ')

In [64]:
script_html

'<script id="__NEXT_DATA__" type="application/json">{"props":{"pageProps":{"type":"loaded","routingProps":{"params":{},"location":{"pathname":"/","query":{},"search":"","hash":""}},"pageQueryVariables":{"cursor":null,"kind":"FEATURED","filters":{}},"_sentryTraceData":"3a9cd6be3f0249eb9ed0995330e88316-9f8643be149a33d2-0","_sentryBaggage":"sentry-environment=production,sentry-release=K77Jpoq1o8Vil1RvwKrJp,sentry-public_key=58db488a6c2941a28833ea5ca522361e,sentry-trace_id=3a9cd6be3f0249eb9ed0995330e88316,sentry-sample_rate=0,sentry-transaction=%2F,sentry-sampled=false"},"serverSideRendered":true,"apolloState":{"Topic44":{"__typename":"Topic","id":"44","slug":"design-tools"},"Topic46":{"__typename":"Topic","id":"46","slug":"productivity"},"Topic352":{"__typename":"Topic","id":"352","slug":"tech"},"FundingSurvey42248":{"__typename":"FundingSurvey","id":"42248","bootstrapped":false},"Product111551":{"__typename":"Product","id":"111551","slug":"pitch","name":"Pitch","postsCount":9,"isSubscrib

In [66]:
script_re = re.compile(r'({.*})__NEXT_DATA__({.*})', flags=re.MULTILINE)
raw_json = script_re.search(script_html).group(1)
parsed = json.loads(raw_json)


AttributeError: 'NoneType' object has no attribute 'group'

In [54]:
parsed

{'a': 'b'}